### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# File to Load (Remember to Change These)
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])



school_data_complete.to_csv("output.csv")

## Local Government Area Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average maths score 

* Calculate the average reading score

* Calculate the percentage of students with a passing maths score (50 or greater)

* Calculate the percentage of students with a passing reading score (50 or greater)

* Calculate the percentage of students who passed maths **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
#calc total schools
num_schools = school_data_complete['school_name'].nunique()
#calc total number of students
num_students = school_data_complete['Student ID'].nunique()
#calc total budget
total_budget = school_data['budget'].sum()

In [3]:
#Calc average math score
avg_math = school_data_complete['maths_score'].mean()
#Calc average reading score
avg_reading = school_data_complete['reading_score'].mean()

In [4]:
#set passing score
pass_score = 50
#calc percentage of students passing math
pass_math = ((student_data['maths_score'] >= pass_score).sum()) / num_students
#calc percentage of students passing math
pass_reading = ((student_data['reading_score'] >= pass_score).sum()) / num_students

#calc percentage of students passing math & Reading
pass_reading_maths = (((student_data['reading_score'] >= pass_score) & (student_data['maths_score'] >= pass_score)).sum()) / num_students

In [5]:
#Create DF

area_summary = pd.DataFrame({
                "Number of Schools": num_schools,
                "Number of Students": num_students,
                "Total Budget": total_budget,
                "Average Math Score": avg_math,
                "Average Reading Score": avg_reading,
                "Percentage Passing Math": pass_math * 100,
                "Percentage Passing Reading": pass_reading * 100,
                "Percentage Passing Math & Reading": pass_reading_maths * 100}
                ,index=[0]) #Need to set an index as all values are singlular

In [6]:
#tidy table output using mapping
area_summary['Number of Students'] = area_summary['Number of Students'].map("{:,}".format)
area_summary['Total Budget'] = area_summary['Total Budget'].map("${:,}".format)
area_summary['Average Math Score'] = area_summary['Average Math Score'].map("{:.2f}".format)
area_summary['Average Reading Score'] = area_summary['Average Reading Score'].map("{:.2f}".format)
area_summary['Percentage Passing Math'] = (area_summary['Percentage Passing Math']).map("{:.2f}%".format)
area_summary['Percentage Passing Reading'] = (area_summary['Percentage Passing Reading']).map("{:.2f}%".format)
area_summary['Percentage Passing Math & Reading'] = (area_summary['Percentage Passing Math & Reading']).map("{:.2f}%".format)
area_summary

,Number of Schools,Number of Students,Total Budget,Average Math Score,Average Reading Score,Percentage Passing Math,Percentage Passing Reading,Percentage Passing Math & Reading
0,15,"39,170","$24,649,428",70.34,69.98,86.08%,84.43%,72.81%


## School Summary

* Create an overview table that summarises key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Maths Score
  * Average Reading Score
  * % Passing Maths
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed maths **and** reading.)
  
* Create a dataframe to hold the above results

In [17]:
# add extra column to school_data_complete for whether student is passing math:
school_data_complete['pass_math'] = school_data_complete['maths_score'] >= pass_score
# add extra column to school_data_complete for whether student is passing reading:
school_data_complete['pass_reading'] = school_data_complete['reading_score'] >= pass_score
# add extra column to school_data_complete for whether student is passing math and reading: 
school_data_complete['pass_reading_math'] = (school_data_complete['pass_math'] & school_data_complete['pass_reading'])

In [18]:
# groupby school and calculate total number of students, size and budget
school_summary = school_data_complete.groupby(['school_name', 'type']).agg({'Student ID': 'count', #total student number
                                                                           'budget': 'mean', #budget per school
                                                                           'reading_score' : 'mean', #avg reading score
                                                                           'maths_score' : 'mean', #avg math score
                                                                            'pass_math': 'sum', #num students pass math
                                                                            'pass_reading': 'sum', #num students pass reading
                                                                            'pass_reading_math': 'sum' #num students pass both
                                                                            })

#Reset indexes:
school_summary = school_summary.reset_index()

In [19]:
school_summary['pass_reading'].sum()

33070

In [10]:
#add column for per student budget
school_summary['Per Student Budget ($)'] = school_summary['budget'] / school_summary['Student ID']

#add column for % Passing Maths
school_summary['Passing Math(%)'] = round((school_summary['pass_math'] / school_summary['Student ID']) * 100,2)

#add column for Passing Reading
school_summary['Passing Reading(%)'] = round((school_summary['pass_reading'] / school_summary['Student ID']) * 100,2)

#add column for Overall Passing (The percentage of students that passed maths **and** reading.)
school_summary['Passing Overall(%)'] = round((school_summary['pass_reading_math'] / school_summary['Student ID']) * 100,2)

#Round reading_score
school_summary['reading_score'] = round(school_summary['reading_score'],2)

#Round maths_score
school_summary['maths_score'] = round(school_summary['maths_score'],2)

In [11]:
#Select only the needed columns
per_school_summary = school_summary.drop(['pass_math', 'pass_reading', 'pass_reading_math'], axis=1)



#Rename columns
per_school_summary = per_school_summary.rename({'school_name': "School Name",
                             'type': 'Type',
                             'reading_score': 'Avg Reading Score',
                             'maths_score': 'Avg Maths Score',
                             'budget': 'Budget($)',
                             'Student ID': 'Number of Students'
                             }, axis = 1)


per_school_summary.head(15)


,School Name,Type,Number of Students,Budget($),Avg Reading Score,Avg Maths Score,Per Student Budget ($),Passing Math(%),Passing Reading(%),Passing Overall(%)
0,Bailey High School,Government,4976,3124928.0,71.01,72.35,628.0,91.64,87.38,80.08
1,Cabrera High School,Independent,1858,1081356.0,71.36,71.66,582.0,90.85,89.07,80.79
2,Figueroa High School,Government,2949,1884411.0,69.08,68.70,639.0,81.65,82.81,67.65
3,Ford High School,Government,2739,1763916.0,69.57,69.09,644.0,82.44,82.22,67.47
4,Griffin High School,Independent,1468,917500.0,71.25,71.79,625.0,91.21,88.49,81.34
5,Hernandez High School,Government,4635,3022020.0,69.19,68.87,652.0,80.95,81.88,66.36
6,Holden High School,Independent,427,248087.0,71.66,72.58,581.0,89.93,88.52,78.92
7,Huang High School,Government,2917,1910635.0,68.91,68.94,655.0,81.69,81.45,66.71
8,Johnson High School,Government,4761,3094650.0,69.04,68.84,650.0,82.06,81.98,67.19
9,Pena High School,Independent,962,585858.0,71.61,72.09,609.0,91.68,86.59,79.21


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [ ]:
#Select top 5 school. nlargest automaticly sorts
top_five_schools = per_school_summary.nlargest(5, 'Passing Overall(%)')
#Display
top_five_schools

#Alternative method: 
#top_five_schools = school_summary_final.loc[school_summary_final['Passing Overall(%)'].sort_values(ascending=False).head(5).index]


,School Name,Type,Number of Students,Budget($),Avg Reading Score,Avg Maths Score,Per Student Budget ($),Passing Math(%),Passing Reading(%),Passing Overall(%)
4,Griffin High School,Independent,1468,917500.0,71.25,71.79,625.0,89.37,86.99,78.27
0,Bailey High School,Government,4976,3124928.0,71.01,72.35,628.0,89.95,85.41,76.95
9,Pena High School,Independent,962,585858.0,71.61,72.09,609.0,90.23,85.24,76.92
14,Wright High School,Independent,1800,1049400.0,70.97,72.05,583.0,90.50,84.50,76.72
1,Cabrera High School,Independent,1858,1081356.0,71.36,71.66,582.0,88.81,86.71,76.64


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [ ]:
#Select top 5 school. nlargest automaticly sorts
bottom_five_schools = per_school_summary.nsmallest(5, 'Passing Overall(%)')
#Display
bottom_five_schools

,School Name,Type,Number of Students,Budget($),Avg Reading Score,Avg Maths Score,Per Student Budget ($),Passing Math(%),Passing Reading(%),Passing Overall(%)
5,Hernandez High School,Government,4635,3022020.0,69.19,68.87,652.0,79.37,79.98,63.73
7,Huang High School,Government,2917,1910635.0,68.91,68.94,655.0,80.25,79.81,64.18
8,Johnson High School,Government,4761,3094650.0,69.04,68.84,650.0,80.30,80.53,64.46
13,Wilson High School,Independent,2283,1319574.0,68.88,69.17,578.0,80.73,79.89,64.70
2,Figueroa High School,Government,2949,1884411.0,69.08,68.70,639.0,80.26,80.74,64.94


## Maths Scores by Year

* Create a table that lists the average maths score for students of each year level (9, 10, 11, 12) at each school.

  * Create a pandas series for each year. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [ ]:
#groupby to create df of school math average scores by year
by_year_maths_avg = school_data_complete.groupby(['year', 'school_name'])['maths_score'].mean()

In [ ]:
#Pull out each year using loc and index (grade number)
year9_math = by_year_maths_avg.loc[9]
year10_math = by_year_maths_avg.loc[10]
year11_math = by_year_maths_avg.loc[11]
year12_math = by_year_maths_avg.loc[12]

In [ ]:
#calc mean math by school
year9_math_by_school = year9_math.groupby('school_name').mean()
year10_math_by_school = year10_math.groupby('school_name').mean()
year11_math_by_school = year11_math.groupby('school_name').mean()
year12_math_by_school = year12_math.groupby('school_name').mean()

In [ ]:
#create df from the year series, round to tidy table 
yearly_math_by_school = pd.DataFrame({'Year 9': round(year9_math_by_school,2),
                                     'Year 10': round(year10_math_by_school,2),
                                     'Year 11': round(year11_math_by_school,2),
                                     'Year 12': round(year12_math_by_school,2)})

#Reset index
yearly_math_by_school.reset_index(inplace=True)

yearly_math_by_school.head()

,school_name,Year 9,Year 10,Year 11,Year 12
0,Bailey High School,72.49,71.90,72.37,72.68
1,Cabrera High School,72.32,72.44,71.01,70.60
2,Figueroa High School,68.48,68.33,68.81,69.33
3,Ford High School,69.02,69.39,69.25,68.62
4,Griffin High School,72.79,71.09,71.69,71.47


## Reading Score by Year

* Perform the same operations as above for reading scores

In [ ]:
#groupby to create df of school reading average scores by year
by_year_reading_avg = school_data_complete.groupby(['year', 'school_name'])['reading_score'].mean()

In [ ]:
#Pull out each year using loc and index (grade number)
year9_reading = by_year_reading_avg.loc[9]
year10_reading = by_year_reading_avg.loc[10]
year11_reading = by_year_reading_avg.loc[11]
year12_reading = by_year_reading_avg.loc[12]

In [ ]:
#calc mean reading by school
year9_reading_by_school = year9_reading.groupby('school_name').mean()
year10_reading_by_school = year10_reading.groupby('school_name').mean()
year11_reading_by_school = year11_reading.groupby('school_name').mean()
year12_reading_by_school = year12_reading.groupby('school_name').mean()

In [ ]:
#create df from the year series 
yearly_reading_by_school = pd.DataFrame({'year 9': year9_reading_by_school,
                                     'year 10': year10_reading_by_school,
                                     'year 11': year10_reading_by_school,
                                     'year 12': year12_reading_by_school})

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Maths Score
  * Average Reading Score
  * % Passing Maths
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [ ]:
#Initiate bin sizes & Labels
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

#Assign summary to new df
school_spending_df = per_school_summary

#use pd.cut() to assign bins
school_spending_df['Spending Ranges (Per Student)'] = pd.cut(school_spending_df['Per Student Budget ($)'], bins=spending_bins, labels=labels)

In [ ]:
#Create series with avg scores and passing scores
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Avg Maths Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Avg Reading Score"].mean()
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Passing Math(%)"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Passing Reading(%)"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Passing Overall(%)"].mean()



In [ ]:
spending_summary = pd.DataFrame({"Average maths score": spending_math_scores,
                                 "Average reading score": spending_reading_scores,
                                 "% passing maths": spending_passing_math,
                                 "% passing reading": spending_passing_reading,
                                 "% overall passing": overall_passing_spending})
spending_summary

,Average maths score,Average reading score,% passing maths,% passing reading,% overall passing
Spending Ranges (Per Student),,,,,
<$585,71.365000,70.717500,87.082500,84.437500,73.367500
$585-630,72.065000,71.032500,89.847500,85.605000,76.960000
$630-645,69.855000,69.840000,83.087500,81.970000,68.230000
$645-680,68.883333,69.046667,79.973333,80.106667,64.123333


## Scores by School Size

* Perform the same operations as above, based on school size.

In [ ]:
#Initiate bin sizes & Labels
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]


In [ ]:
#Use pd.cut to assign bins
per_school_summary['School Size'] = pd.cut(per_school_summary['Number of Students'], bins=size_bins, labels=labels)


In [ ]:
#Create series with avg scores and passing scores
size_math_scores = per_school_summary.groupby(["School Size"])["Avg Maths Score"].mean()
size_reading_scores = per_school_summary.groupby(["School Size"])["Avg Reading Score"].mean()
size_passing_math = per_school_summary.groupby(["School Size"])["Passing Math(%)"].mean()
size_passing_reading = per_school_summary.groupby(["School Size"])["Passing Reading(%)"].mean()
overall_passing_size = per_school_summary.groupby(["School Size"])["Passing Overall(%)"].mean()

In [ ]:
#Create df from scores
size_summary = pd.DataFrame({"Average maths score": size_math_scores,
                                 "Average reading score": size_reading_scores,
                                 "% passing maths": size_passing_math,
                                 "% passing reading": size_passing_reading,
                                 "% overall passing": overall_passing_size})
size_summary

,Average maths score,Average reading score,% passing maths,% passing reading,% overall passing
School Size,,,,,
Small (<1000),72.33500,71.6350,89.26,85.9450,76.165
Medium (1000-2000),71.42200,70.7220,88.07,84.7560,74.678
Large (2000-5000),69.75125,69.5775,82.64,81.5875,67.610


## Scores by School Type

* Perform the same operations as above, based on school type

In [ ]:
#Create series with avg scores and passing scores
type_math_scores = per_school_summary.groupby(["Type"])["Avg Maths Score"].mean()
type_reading_scores = per_school_summary.groupby(["Type"])["Avg Reading Score"].mean()
type_passing_math = per_school_summary.groupby(["Type"])["Passing Math(%)"].mean()
type_passing_reading = per_school_summary.groupby(["Type"])["Passing Reading(%)"].mean()
overall_passing_type = per_school_summary.groupby(["Type"])["Passing Overall(%)"].mean()

In [ ]:
#Create df from scores
type_summary = pd.DataFrame({"Average maths score": type_math_scores,
                                 "Average reading score": type_reading_scores,
                                 "% passing maths": type_passing_math,
                                 "% passing reading": type_passing_reading,
                                 "% overall passing": overall_passing_type})
type_summary

,Average maths score,Average reading score,% passing maths,% passing reading,% overall passing
Type,,,,,
Government,69.834286,69.677143,82.912857,81.830,68.025714
Independent,71.368750,70.720000,87.450000,84.445,73.802500
